# Item KNN Incremental Algorithm on Test Dataset

In [ ]:
k = 2
from streamsightv2.datasets import TestDataset
from streamsightv2.settings import SlidingWindowSetting
dataset = TestDataset()
data = dataset.load()
setting_window = SlidingWindowSetting(
    4,
    3,
    0,
    k
)
setting_window.split(data)

INFO - streamsight package loaded.
WARNING - /Users/joshua/projects/fyp/streamsightv2/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

DEBUG - TestDataset being initialized with 'data' as the base path.
DEBUG - TestDataset is initialized.
INFO - TestDataset is loading dataset...
DEBUG - TestDataset applying filters set.
DEBUG - 	interactions before preprocess: 13
DEBUG - 	items before preprocess: 3
DEBUG - 	users before preprocess: 5
DEBUG - 	interactions after preprocess: 13
DEBUG - 	items after preprocess: 3
DEBUG - 	users after preprocess: 5
INFO - TestDataset dataset loaded - Took 0.0129s
DEBUG - Splitting data...
DEBUG - Performing lt(t, 2147483647)
DEBUG - Performing lt(t, 4)
DEBUG - Performing ge(t, 4)
DEBUG - TimestampSplitter(t=4,t_lower=None,t_upper=None) has complete split


  0%|          | 0/2 [00:00<?, ?it/s]

DEBUG - NPastInteractionTimestampSplitter(t=4,t_lower=None,t_upper=3,n_seq_data=0,include_all_past_data=False) - Updating split point to t=4
DEBUG - Performing lt(t, 7)
DEBUG - Performing ge(t, 4)
DEBUG - Performing get_user_n_last_interaction comparison
DEBUG - NPastInteractionTimestampSplitter(t=4,t_lower=None,t_upper=3,n_seq_data=0,include_all_past_data=False) has complete split
INFO - Split at time 4 resulted in empty unlabelled testing samples.
DEBUG - NPastInteractionTimestampSplitter(t=4,t_lower=None,t_upper=3,n_seq_data=0,include_all_past_data=False) - Updating split point to t=7
DEBUG - Performing lt(t, 10)
DEBUG - Performing ge(t, 7)
DEBUG - Performing get_user_n_last_interaction comparison
DEBUG - NPastInteractionTimestampSplitter(t=7,t_lower=None,t_upper=3,n_seq_data=0,include_all_past_data=False) has complete split
INFO - Split at time 7 resulted in empty unlabelled testing samples.
DEBUG - NPastInteractionTimestampSplitter(t=7,t_lower=None,t_upper=3,n_seq_data=0,include_a

3it [00:00, 61.02it/s]               

INFO - Finished split with window size 3 seconds. Number of splits: 3 in total.
INFO - SlidingWindowSetting data split - Took 0.0852s
DEBUG - Checking split attribute and sizes.
DEBUG - Checking split attributes.
DEBUG - Split attributes are set.
DEBUG - Checking size of split sets.
DEBUG - Size of split sets are checked.
INFO - SlidingWindowSetting data split complete.


In [2]:
data._df

,interactionid,uid,iid,ts,rating
0,0,0,0,0,1
1,1,1,0,1,2
2,2,2,1,2,3
3,3,0,2,3,4
4,4,1,1,4,5
5,5,1,2,5,1
6,6,3,1,6,2
7,7,2,0,7,3
8,8,2,2,8,4
9,9,3,2,9,5


In [ ]:
from streamsightv2.evaluators import EvaluatorStreamerBuilder

builder = EvaluatorStreamerBuilder()
builder.add_setting(setting_window)
builder.set_metric_K(k)
# builder.add_metric("PrecisionK")
builder.add_metric("RecallK")
evaluator = builder.build()

In [ ]:
from streamsightv2.algorithms import ItemKNNIncremental
# from recpack.algorithms import Popularity
external_model = ItemKNNIncremental(K=2)
# external_model = ItemKNNIncremental(K=10)
external_model_id = evaluator.register_algorithm(algorithm_name="itemknn_incremental")
print(external_model_id)

INFO - Registering algorithm name itemknn_incremental with ID: bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d registered
bdd640fb-0667-4ad1-9c80-317fa3b1799d


In [5]:
evaluator.start_stream()

DEBUG - Resetting data generators.
DEBUG - Data generators are reset.
DEBUG - Preparing evaluator for streaming
DEBUG - (user x item) shape defined is (3, 3)
DEBUG - Shape of dataframe stored in matrix was (4, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (4, 5) after masking
DEBUG - Final (user x item) shape defined is (3, 3)
DEBUG - Caching evaluation data for step 1
DEBUG - (user x item) shape defined is (3, 3)
DEBUG - Shape of dataframe stored in matrix was (3, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (3, 5) after masking
DEBUG - Final (user x item) shape defined is (3, 3)
DEBUG - (user x item) shape defined is (3, 3)
DEBUG - Shape of dataframe stored in matrix was (3, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (3, 5) after masking
DEBUG - Final (user x item) shape defined is (4, 3)
DEBUG - Data cached for step 1 complete


# Iteration 1

In [6]:
data = evaluator.get_data(external_model_id)
external_model.fit(data)
data

DEBUG - Getting data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
Training data:  [[1 0 1]
 [1 0 0]
 [0 1 0]]
Item similarities:  [[0.         0.         0.70710678]
 [0.         0.         0.        ]
 [0.70710678 0.         0.        ]]
Item similarities after get_top_K_values:  [[0.         0.         0.70710678]
 [0.         0.         0.        ]
 [0.70710678 0.         0.        ]]
WARNING - /Users/joshua/projects/fyp/streamsightv2/venv/lib/python3.12/site-packages/streamsight/algorithms/base.py:315: UserWarning: ItemKNNIncremental missing similar items for 1 items.
  warn(f"{self.name} missing similar items for {missing} items.")

DEBUG - Fitting ItemKNNIncremental complete - Took 0.0065s


   interactionid  uid  iid  ts  rating
0              0    0    0   0       1
1              1    1    0   1       2
2              2    2    1   2       3
3              3    0    2   3       4

In [7]:
ul_data = evaluator.get_unlabeled_data(external_model_id)
prediction = external_model.predict(data, ul_data)
ul_data

DEBUG - Getting unlabeled data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
WARNING - /Users/joshua/projects/fyp/streamsightv2/venv/lib/python3.12/site-packages/streamsight/algorithms/base.py:315: UserWarning: ItemKNNIncremental missing similar items for 1 items.
  warn(f"{self.name} missing similar items for {missing} items.")

DEBUG - Performing items_in comparison
In ItemKNNIncremental _predict:  [[0.70710678 0.         0.70710678]
 [0.         0.         0.70710678]
 [0.         0.         0.        ]]
Max user ID:  4
Max item ID:  0
X.shape:  (3, 3)
Intended shape:  (4, 3)
Predict frame:     interactionid  uid  iid  ts  rating
4              4    1   -1   4       5
5              5    1   -1   5       1
6              6    3   -1   6       2
Known user ID:  3
Known item ID:  3
X_pred after adding rows:  [[0.70710678 0.         0.70710678]
 [0.         0.         0.70710678]
 [0.         0.         0.        ]
 [0.         0.         0.        ]]
DEBUG - Padding user ID in ra

   interactionid  uid  iid  ts  rating
4              4    1   -1   4       5
5              5    1   -1   5       1
6              6    3   -1   6       2

In [8]:
print(prediction)
print(prediction.toarray())
evaluator.submit_prediction(external_model_id, prediction)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6 stored elements and shape (4, 3)>
  Coords	Values
  (0, 0)	0.7071067811865475
  (0, 2)	0.7071067811865475
  (1, 2)	0.7071067811865475
  (3, 0)	1.0
  (3, 1)	0.0
  (3, 2)	0.5
[[0.70710678 0.         0.70710678]
 [0.         0.         0.70710678]
 [0.         0.         0.        ]
 [1.         0.         0.5       ]]
DEBUG - Submitting prediction for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Precision compute started - RecallK_2
DEBUG - Number of users: 2
DEBUG - Number of ground truth interactions: 3
DEBUG - Recall compute complete - RecallK_2
DEBUG - Metric RecallK(timestamp_limit=4,K=2) created for algorithm itemknn_incremental_bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Prediction evaluated for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d complete


# Iteration 2

In [9]:
data = evaluator.get_data(external_model_id)
external_model.fit(data)
data

DEBUG - Getting data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - (user x item) shape defined is (4, 3)
DEBUG - Shape of dataframe stored in matrix was (3, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (3, 5) after masking
DEBUG - Final (user x item) shape defined is (4, 3)
DEBUG - Caching evaluation data for step 2
DEBUG - (user x item) shape defined is (4, 3)
DEBUG - Shape of dataframe stored in matrix was (4, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (4, 5) after masking
DEBUG - Final (user x item) shape defined is (4, 3)
DEBUG - (user x item) shape defined is (4, 3)
DEBUG - Shape of dataframe stored in matrix was (4, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (4, 5) after masking
DEBUG - Final (user x item) shape defined is (5, 3)
DEBUG - Data cached for step 2 complete
Training data:  [[1. 0. 1.]
 [1. 1. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]
Item similarities:  [[0.         0.40824829 1.        ]
 [0.

   interactionid  uid  iid  ts  rating
4              4    1    1   4       5
5              5    1    2   5       1
6              6    3    1   6       2

In [10]:
ul_data = evaluator.get_unlabeled_data(external_model_id)
prediction = external_model.predict(data, ul_data)
ul_data

DEBUG - Getting unlabeled data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Performing items_in comparison
In ItemKNNIncremental _predict:  [[1.         0.81649658 1.        ]
 [1.40824829 0.81649658 1.40824829]
 [0.40824829 0.         0.40824829]
 [0.40824829 0.         0.40824829]]
Max user ID:  5
Max item ID:  0
X.shape:  (4, 3)
Intended shape:  (5, 3)
Predict frame:      interactionid  uid  iid  ts  rating
7               7    2   -1   7       3
8               8    2   -1   8       4
9               9    3   -1   9       5
10             10    4   -1   9       1
Known user ID:  4
Known item ID:  3
X_pred after adding rows:  [[1.         0.81649658 1.        ]
 [1.40824829 0.81649658 1.40824829]
 [0.40824829 0.         0.40824829]
 [0.40824829 0.         0.40824829]
 [0.         0.         0.        ]]
DEBUG - Padding user ID in range(4, 5) with items
To predict:  uid
2    2
3    1
4    1
Name: count, dtype: int64
Popular items:  [0.  1.  0.5]
WARNING - /Users/joshua/

    interactionid  uid  iid  ts  rating
7               7    2   -1   7       3
8               8    2   -1   8       4
9               9    3   -1   9       5
10             10    4   -1   9       1

In [11]:
print(prediction)
print(prediction.toarray())
evaluator.submit_prediction(external_model_id, prediction)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 13 stored elements and shape (5, 3)>
  Coords	Values
  (0, 0)	0.9999999999999998
  (0, 1)	0.816496580927726
  (0, 2)	0.9999999999999998
  (1, 0)	1.4082482904638627
  (1, 1)	0.816496580927726
  (1, 2)	1.4082482904638627
  (2, 0)	0.408248290463863
  (2, 2)	0.408248290463863
  (3, 0)	0.408248290463863
  (3, 2)	0.408248290463863
  (4, 0)	0.0
  (4, 1)	1.0
  (4, 2)	0.5
[[1.         0.81649658 1.        ]
 [1.40824829 0.81649658 1.40824829]
 [0.40824829 0.         0.40824829]
 [0.40824829 0.         0.40824829]
 [0.         1.         0.5       ]]
DEBUG - Submitting prediction for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Precision compute started - RecallK_2
DEBUG - Number of users: 3
DEBUG - Number of ground truth interactions: 4
DEBUG - Recall compute complete - RecallK_2
DEBUG - Metric RecallK(timestamp_limit=7,K=2) created for algorithm itemknn_incremental_bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Prediction ev

# Iteration 3

In [12]:
data = evaluator.get_data(external_model_id)
external_model.fit(data)
data

DEBUG - Getting data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - (user x item) shape defined is (5, 3)
DEBUG - Shape of dataframe stored in matrix was (4, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (4, 5) after masking
DEBUG - Final (user x item) shape defined is (5, 3)
DEBUG - Caching evaluation data for step 3
DEBUG - (user x item) shape defined is (5, 3)
DEBUG - Shape of dataframe stored in matrix was (2, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (2, 5) after masking
DEBUG - Final (user x item) shape defined is (5, 3)
DEBUG - (user x item) shape defined is (5, 3)
DEBUG - Shape of dataframe stored in matrix was (2, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (2, 5) after masking
DEBUG - Final (user x item) shape defined is (5, 3)
DEBUG - Data cached for step 3 complete
Training data:  [[1. 0. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 [0. 1. 1.]
 [1. 0. 0.]]
Item similarities:  [[0.         0.57735027 0.75

    interactionid  uid  iid  ts  rating
7               7    2    0   7       3
8               8    2    2   8       4
9               9    3    2   9       5
10             10    4    0   9       1

In [13]:
ul_data = evaluator.get_unlabeled_data(external_model_id)
prediction = external_model.predict(data, ul_data)
ul_data

DEBUG - Getting unlabeled data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Performing items_in comparison
In ItemKNNIncremental _predict:  [[0.75       1.44337567 0.75      ]
 [1.32735027 1.44337567 1.6160254 ]
 [1.32735027 1.44337567 1.6160254 ]
 [1.32735027 0.8660254  0.8660254 ]
 [0.         0.57735027 0.75      ]]
Max user ID:  5
Max item ID:  0
X.shape:  (5, 3)
Intended shape:  (5, 3)
Predict frame:      interactionid  uid  iid  ts  rating
11             11    4   -1  10       2
12             12    4   -1  10       3


    interactionid  uid  iid  ts  rating
11             11    4   -1  10       2
12             12    4   -1  10       3

In [14]:
print(prediction)
print(prediction.toarray())
evaluator.submit_prediction(external_model_id, prediction)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 14 stored elements and shape (5, 3)>
  Coords	Values
  (0, 0)	0.75
  (0, 1)	1.4433756729740645
  (0, 2)	0.75
  (1, 0)	1.3273502691896257
  (1, 1)	1.4433756729740645
  (1, 2)	1.6160254037844388
  (2, 0)	1.3273502691896257
  (2, 1)	1.4433756729740645
  (2, 2)	1.6160254037844388
  (3, 1)	0.8660254037844388
  (3, 0)	1.3273502691896257
  (3, 2)	0.8660254037844388
  (4, 1)	0.5773502691896258
  (4, 2)	0.75
[[0.75       1.44337567 0.75      ]
 [1.32735027 1.44337567 1.6160254 ]
 [1.32735027 1.44337567 1.6160254 ]
 [1.32735027 0.8660254  0.8660254 ]
 [0.         0.57735027 0.75      ]]
DEBUG - Submitting prediction for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Precision compute started - RecallK_2
DEBUG - Number of users: 1
DEBUG - Number of ground truth interactions: 2
DEBUG - Recall compute complete - RecallK_2
DEBUG - Metric RecallK(timestamp_limit=10,K=2) created for algorithm itemknn_incremental_bdd640fb-0667-4ad1-9

# evaluate metrics

In [15]:
evaluator.get_all_algorithm_status()

{'itemknn_incremental_bdd640fb-0667-4ad1-9c80-317fa3b1799d': <AlgorithmStateEnum.COMPLETED: 'COMPLETED'>}

In [16]:
evaluator.metric_results("macro")

,,macro_score,num_window
Algorithm,Metric,,
itemknn_incremental_bdd640fb-0667-4ad1-9c80-317fa3b1799d,RecallK_2,0.638889,3


In [17]:
evaluator.metric_results("micro")


,,micro_score,num_user
Algorithm,Metric,,
itemknn_incremental_bdd640fb-0667-4ad1-9c80-317fa3b1799d,RecallK_2,0.583333,6


In [18]:
evaluator.metric_results("user")

user_id  \
Algorithm                                          Timestamp Metric              
itemknn_incremental_bdd640fb-0667-4ad1-9c80-317... t=4       RecallK_2       1   
                                                             RecallK_2       3   
                                                   t=7       RecallK_2       2   
                                                             RecallK_2       3   
                                                             RecallK_2       4   
                                                   t=10      RecallK_2       4   

                                                                       score  
Algorithm                                          Timestamp Metric           
itemknn_incremental_bdd640fb-0667-4ad1-9c80-317... t=4       RecallK_2   0.5  
                                                             RecallK_2   0.0  
                                                   t=7       RecallK_2   1.0  
                                                             RecallK_2   1.0  
                                                             RecallK_2   0.0  
                                                   t=10      RecallK_2   1.0

In [19]:
evaluator.metric_results("window")

window_score  \
Algorithm                                          Timestamp Metric                    
itemknn_incremental_bdd640fb-0667-4ad1-9c80-317... t=4       RecallK_2      0.250000   
                                                   t=7       RecallK_2      0.666667   
                                                   t=10      RecallK_2      1.000000   

                                                                        num_user  
Algorithm                                          Timestamp Metric               
itemknn_incremental_bdd640fb-0667-4ad1-9c80-317... t=4       RecallK_2         2  
                                                   t=7       RecallK_2         3  
                                                   t=10      RecallK_2         1